<a href="https://colab.research.google.com/github/philschmid/efsync/blob/main/examples/efsync_pip_packages_and_s3_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Notebook for efsync installing pip packages and syncing files from s3 to efs

this notebooks includes examples for the `CLI` and the `SDK`. 

This Notebook can be runned in google colab. 

In [2]:
!pip3 install efsync

     |████████████████████████████████| 215kB 5.4MB/s 
     |████████████████████████████████| 962kB 8.1MB/s 
     |████████████████████████████████| 2.6MB 14.2MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 


In [15]:
%cd ..
!python3 setup.py sdist bdist_wheel
! pip install dist/efsync-0.9.24-py3-none-any.whl 
%cd examples 

/Users/philippschmid/projects/personal/efsync
['efsync', 'efsync.test', 'efsync.utils', 'efsync.utils.config', 'efsync.utils.security_group', 'efsync.utils.ssh', 'efsync.utils.iam_profile', 'efsync.utils.efs', 'efsync.utils.ec2']
running sdist
running egg_info
writing efsync.egg-info/PKG-INFO
writing dependency_links to efsync.egg-info/dependency_links.txt
writing entry points to efsync.egg-info/entry_points.txt
writing requirements to efsync.egg-info/requires.txt
writing top-level names to efsync.egg-info/top_level.txt
reading manifest file 'efsync.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'efsync.egg-info/SOURCES.txt'
running check
creating efsync-0.9.24
creating efsync-0.9.24/efsync
creating efsync-0.9.24/efsync.egg-info
creating efsync-0.9.24/efsync/test
creating efsync-0.9.24/efsync/utils
creating efsync-0.9.24/efsync/utils/config
creating efsync-0.9.24/efsync/utils/ec2
creating efsync-0.9.24/efsync/utils/efs
creating efsync-0.9.24/efsync/

Processing ./dist/efsync-0.9.24-py3-none-any.whl
/Users/philippschmid/projects/personal/efsync/examples


In [14]:
!pip uninstall efsync -y

Found existing installation: efsync 0.9.22
Uninstalling efsync-0.9.22:
  Successfully uninstalled efsync-0.9.22


# Check installed version

In [16]:
!pip freeze | grep efsync

efsync==0.9.24


In [5]:
!efsync -h

usage: efsync <command> [<args>]

optional arguments:
  -h, --help            show this help message and exit
  --version, -v         show program version
  --requirements REQUIREMENTS, -r REQUIREMENTS
                        path of your requirements.txt
  --config_file CONFIG_FILE, -cf CONFIG_FILE
                        path of your efsync.yaml
  --python_version PYTHON_VERSION, -py PYTHON_VERSION
                        Python version used to install dependencies
  --efs_pip_dir EFS_PIP_DIR, -epd EFS_PIP_DIR
                        directory where the pip packages will be installed on
                        efs
  --efs_filesystem_id EFS_FILESYSTEM_ID, -efi EFS_FILESYSTEM_ID
                        File System ID from the EFS filesystem
  --clean_efs CLEAN_EFS, -ce CLEAN_EFS
                        Defines if the efs should be cleaned up before
                        uploading
  --file_dir FILE_DIR, -fd FILE_DIR
                        directory where all other files will be place

In [6]:
from efsync import efsync

---

## Create `requirements.txt` 

In [7]:
%%writefile ./requirements.txt

pyjokes
pandas
numpy
requests

Overwriting ./requirements.txt


# Example with `efsync.yaml`

In [8]:
%%writefile ./efsync.yaml

#standard configuration
efs_filesystem_id: fs-2226b27a # aws efs filesystem id (moint point)
subnet_Id: subnet-17f97a7d # subnet of which the efs is running in
ec2_key_name: efsync-asd913fjgq3 # required key name for starting the ec2 instance
clean_efs: all # Defines if the EFS should be cleaned up before. values: `'all'`,`'pip'`,`'file'` uploading
# aws profile configuration
aws_profile: schueler-vz # aws iam profile with required permission configured in .aws/credentials
aws_region: eu-central-1 # the aws region where the efs is running

# pip packages configurations
efs_pip_dir: lib # pip directory on ec2
python_version: 3.8 # python version used for installing pip packages -> should be used as lambda runtime afterwads
requirements: requirements.txt # path + file to requirements.txt which holds the installable pip packages
# s3 config
s3_bucket: efsync-test-bucket # s3 bucket name from files should be downloaded
s3_keyprefix: distilbert # s3 keyprefix for the files
file_dir_on_ec2: ml # name of the directory where your file from <file_dir> will be uploaded

Overwriting ./efsync.yaml


In [9]:
from efsync import efsync

efsync('./efsync.yaml')

2020-10-25 20:12:33,747 - efsync - starting....
2020-10-25 20:12:33,748 - efsync - loading config
2020-10-25 20:12:33,772 - efsync - creating security group
2020-10-25 20:12:34,379 - efsync - loading default security group
2020-10-25 20:12:39,444 - efsync - creating ssh key for scp in memory
2020-10-25 20:12:40,005 - efsync - starting ec2 instance with security group sg-0ff6539317d7e48da and subnet_Id subnet-17f97a7d
2020-10-25 20:18:46,430 - efsync - stopping ec2 instance with instance id i-020e3f3cc4b3d690b
2020-10-25 20:19:17,159 - efsync - deleting iam profile
2020-10-25 20:19:18,354 - efsync - deleting ssh key
2020-10-25 20:19:18,604 - efsync - deleting security group
2020-10-25 20:19:18,914 - efsync - #################### finished after 6.752833333333333 minutes ####################


True

# Example with pyton `dict`

In [10]:
config = {
  'efs_filesystem_id': 'fs-2226b27a', # aws efs filesystem id (moint point)
  'subnet_Id': 'subnet-17f97a7d', # subnet of which the efs is running in
  'ec2_key_name':'efsync-asd913fjgq3',  # required key name for starting the ec2 instance
  'clean_efs': 'all', # Defines if the EFS should be cleaned up before. values: `'all'`,`'pip'`,`'file'` uploading
  'aws_profile': 'schueler-vz', # aws iam profile with required permission configured in .aws/credentials
  'aws_region': 'eu-central-1', # the aws region where the efs is running
  'efs_pip_dir': 'lib',  # pip directory on ec2
  'python_version': 3.8,  # python version used for installing pip packages -> should be used as lambda runtime afterwads
  'requirements': 'requirements.txt', # path + file to requirements.txt which holds the installable pip packages
  's3_bucket': 'efsync-test-bucket', # s3 bucket name from files should be downloaded
  's3_keyprefix': 'distilbert', # s3 keyprefix for the files
  'file_dir_on_ec2': 'ml' # name of the directory where your file from <file_dir> will be uploaded
}

In [17]:
from efsync import efsync


efsync(config)

2020-10-25 21:02:26,188 - efsync - starting....
2020-10-25 21:02:26,189 - efsync - loading config
2020-10-25 21:02:26,205 - efsync - creating security group
2020-10-25 21:02:26,900 - efsync - loading default security group
2020-10-25 21:02:27,162 - efsync - creating ssh key for scp in memory
2020-10-25 21:02:27,408 - efsync - starting ec2 instance with security group sg-009e0c433aae54ad3 and subnet_Id subnet-17f97a7d
2020-10-25 21:09:14,586 - efsync - stopping ec2 instance with instance id i-0a54cf19e1cac5898
2020-10-25 21:09:45,663 - efsync - deleting iam profile
2020-10-25 21:09:46,896 - efsync - deleting ssh key
2020-10-25 21:09:47,176 - efsync - deleting security group
2020-10-25 21:09:47,676 - efsync - #################### finished after 7.3581666666666665 minutes ####################


True

# Example with cli and `efsync.yaml`

In [12]:
!efsync -cf efsync.yaml

2020-10-25 20:29:45,885 - efsync - starting....
2020-10-25 20:29:45,885 - efsync - loading config
2020-10-25 20:29:45,895 - efsync - creating security group
2020-10-25 20:29:46,482 - efsync - loading default security group
2020-10-25 20:29:46,814 - efsync - creating ssh key for scp in memory
2020-10-25 20:29:47,943 - efsync - starting ec2 instance with security group sg-082ab943c8ddc652c and subnet_Id subnet-17f97a7d
2020-10-25 20:36:18,962 - efsync - stopping ec2 instance with instance id i-0773347af1937b71b
2020-10-25 20:36:50,055 - efsync - deleting iam profile
2020-10-25 20:36:51,210 - efsync - deleting ssh key
2020-10-25 20:36:51,438 - efsync - deleting security group
2020-10-25 20:36:51,751 - efsync - #################### finished after 7.097833333333333 minutes ####################


# Example with cli and parameters

In [13]:
!efsync --efs_filesystem_id  fs-2226b27a \
        --subnet_Id subnet-17f97a7d \
        --ec2_key_name efsync-asd913fjgq3 \
        --clean_efs all \
        --aws_profile schueler-vz \
        --aws_region eu-central-1 \
        --efs_pip_dir lib \
        --python_version 3.8 \
        --requirements requirements.txt \
        --s3_bucket efsync-test-bucket \
        --s3_keyprefix distilbert \
        --file_dir_on_ec2 ml

2020-10-25 20:37:38,406 - efsync - starting....
2020-10-25 20:37:38,406 - efsync - loading config
2020-10-25 20:37:38,415 - efsync - creating security group
2020-10-25 20:37:38,999 - efsync - loading default security group
2020-10-25 20:37:39,258 - efsync - creating ssh key for scp in memory
2020-10-25 20:37:39,442 - efsync - starting ec2 instance with security group sg-040231be49803d280 and subnet_Id subnet-17f97a7d
2020-10-25 20:43:56,074 - efsync - stopping ec2 instance with instance id i-013fd81d67e42c534
2020-10-25 20:44:29,590 - efsync - deleting iam profile
2020-10-25 20:44:34,379 - efsync - deleting ssh key
2020-10-25 20:44:38,226 - efsync - deleting security group
2020-10-25 20:44:39,858 - efsync - #################### finished after 7.024166666666667 minutes ####################
